# Initialisation

In [1]:
import sys
import notebook
import re
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier


In [2]:
packages = {
    "Python": sys,
    "Jupyter Notebook": "notebook",
    "NumPy": "numpy",
    "Pandas": "pandas",
    "Matplotlib": "matplotlib",
    "Seaborn": "seaborn",
    "MLflow" : "mlflow",
    "Scikit-Learn": "sklearn",
    "LightGMB": "lightgbm"
}

In [3]:
errorMsg = (
    "non disponible - vérifiez que le package existe, qu'il est correctement installé, importé "
    "et qu'il dispose d'un attribut '__version__'."
)
for name, module in packages.items():
    if isinstance(module, str):
        version = getattr(sys.modules.get(module, None), '__version__', errorMsg)
    else:
        if module is sys:
            version = sys.version
        else:
            version = getattr(module, '__version__', errorMsg)

    print(f"Version de {name} : {version}")

Version de Python : 3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:28:07) [MSC v.1916 64 bit (AMD64)]
Version de Jupyter Notebook : 7.0.8
Version de NumPy : 1.26.4
Version de Pandas : 2.2.3
Version de Matplotlib : 3.9.2
Version de Seaborn : 0.13.2
Version de MLflow : 2.19.0
Version de Scikit-Learn : 1.5.1
Version de LightGMB : 4.3.0


In [7]:
def read_csv_with_fallback(filepath, encodings=['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']):
    """
    Lecture d'un fichier CSV avec tentative sur plusieurs encodages.

    Parameters:
    - filepath (str): Chemin du fichier à lire.
    - encodings (list): Liste des encodages à tester.

    Returns:
    - pd.DataFrame: Le DataFrame pandas chargé.

    Raises:
    - UnicodeDecodeError: Si aucun des encodages ne fonctionne.
    """
    for encoding in encodings:
        try:
            print(f"Tentative de lecture avec l'encodage : {encoding}")
            return pd.read_csv(filepath, encoding=encoding)
        except UnicodeDecodeError:
            print(f"Erreur avec l'encodage : {encoding}")

    raise UnicodeDecodeError(f"Impossible de lire le fichier {filepath} avec les encodages {encodings}")

# Chemin vers le nouveau dossier "data"
data_path = "D://Pro//OpenClassrooms//Projet_7//data//"

# Liste des fichiers et noms de DataFrame correspondants
files = {
    "application_test": "application_test.csv",
    "application_train": "application_train.csv",
    "bureau": "bureau.csv",
    "bureau_balance": "bureau_balance.csv",
    "credit_card_balance": "credit_card_balance.csv",
    "homecredit_columns_description": "HomeCredit_columns_description.csv",
    "installments_payments": "installments_payments.csv",
    "pos_cash_balance": "POS_CASH_balance.csv",
    "previous_application": "previous_application.csv"
}

# Chargement des fichiers dans des DataFrames
loaded_data = {}
for name, file in files.items():
    filepath = data_path + file
    loaded_data[name] = read_csv_with_fallback(filepath)
    print(f"{name} chargé avec succès !\n")

# Affichage des informations pour chaque DataFrame
for name, df in loaded_data.items():
    print(f"{name.capitalize()} Info:")
    print(df.info(), "\n")

Tentative de lecture avec l'encodage : utf-8
application_test chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
application_train chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
bureau chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
bureau_balance chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
credit_card_balance chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
Erreur avec l'encodage : utf-8
Tentative de lecture avec l'encodage : latin1
homecredit_columns_description chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
installments_payments chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
pos_cash_balance chargé avec succès !

Tentative de lecture avec l'encodage : utf-8
previous_application chargé avec succès !

Application_test Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BU

# Préparation de l'environement

In [4]:
# Donnée fictives pour tester le fonctionnement de l'environement
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")  # Stockage local MLflow

with mlflow.start_run(run_name="LightGBM_baseline"):
    # 1. Logging de paramètres basiques
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # 2. Entraînement de LightGBM (sans hyperparamètres particuliers)
    model = LGBMClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 3. Évaluation
    auc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    mlflow.log_metric("AUC", auc_score)

    # 4. Enregistrement du modèle
    mlflow.sklearn.log_model(model, "model")

[LightGBM] [Info] Number of positive: 362, number of negative: 388
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000540 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2502
[LightGBM] [Info] Number of data points in the train set: 750, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.482667 -> initscore=-0.069361
[LightGBM] [Info] Start training from score -0.069361
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

2025/01/10 16:37:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [6]:
# Ouvrir le dossier
# cd /D D:\Pro\OpenClassrooms\Projet_7\3_dossier_code_012025
# Remote
# git remote -v
# git push -u origin main
# davfgh
# token à la place du password git

# Test unitaire
# pytest tests/
# Vérifier dans quel notebook ou script se font les test
# écrire le code qui permet juste de vérifier que cela marche
# Faire les installations
# Créée le dossier

#